### Finding best Minimally Complex model for supreme court voting dataset using integer representation

In [1]:
# Imports
import sys
sys.path.insert(0, '../')
from src.utils import *
from src.mcm_binary import mcm

In [2]:
model = mcm('../data/SC_voting/US_SupremeCourt_n9_N895.txt')

In [3]:
# Evidence of IM with original basis (Should be -5258)
print('Independent model in the original basis')
print('Evidence: ', model.calc_log_evidence(model.mcms[-1]))

Independent model in the original basis
Evidence:  -5258.100240438084


In [4]:
# Finding best MCM in original basis (Should be [[0, 2, 3, 4, 6], [1, 5, 7, 8]] with evidence -3300.4)
model.find_best_mcm()
print('Best MCM in original basis (Exhaustive search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in original basis (Exhaustive search)
MCM:  [[0, 2, 3, 4, 6], [1, 5, 7, 8]]
Evidence:  -3300.395469673639


In [5]:
# Finding best MCM in original basis using a Greedy search (Should be [[0, 2, 3, 4, 6], [1, 5, 7, 8]] with evidence -3300.4)
model.find_best_mcm(method='greedy')
print('Best MCM in original basis (Greedy search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in original basis (Greedy search)
MCM:  [[0, 4, 2, 3, 6], [1, 5, 7, 8]]
Evidence:  -3300.395469673639


In [6]:
# Finding best MCM in original basis using the divide and conquer scheme
model.find_best_mcm(method='divide_and_conquer')
print('Best MCM in original basis (Divide and conquer)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in original basis (Divide and conquer)
MCM:  [[0, 1, 2, 3, 4, 5, 6, 7, 8]]
Evidence:  -3305.54657591369


In [3]:
# Finding the best IM (Should be [72, 160, 384, 17, 65, 5, 130, 260, 64] with evidence -3327)
model.find_best_im(max_interactions=2)
print('Best Independent model')
print('IM: ', model.best_im)
print('Evidence: ', model.calc_log_evidence(model.mcms[-1]))

Best Independent model
IM:  [ 72 160 384  17  65   5 130 260  64]
Evidence:  -3327.079945846724


In [10]:
# Finding best MCM in optimal basis (Should be [[0], [1, 2, 6], [3, 4, 5, 7, 8]] with evidence -3154.4)
model.find_best_mcm()
print('Best MCM in the optimal basis (Exhaustive search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in the optimal basis (Exhaustive search)
MCM:  [[0], [1, 2, 6], [3, 4, 5, 7, 8]]
Evidence:  -3154.421230299754


In [11]:
# Finding the best MCM in the optimal basis using Greedy search ([[0, 4, 3, 5], [1, 2, 6, 7, 8]] -3163.252527450342)
model.find_best_mcm(method='greedy')
print('Best MCM in the optimal basis (Greedy search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in the optimal basis (Greedy search)
MCM:  [[0, 4, 3, 5], [1, 2, 6, 7, 8]]
Evidence:  -3163.252527450342


In [5]:
# Finding the best MCM in the optimal basis using the divide and conquer scheme
model.find_best_mcm(method='divide_and_conquer')
print('Best MCM in the optimal basis (Divide and conquer)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in the optimal basis (Divide and conquer)
MCM:  [[1, 2, 4, 5, 6, 7, 8], [0, 3]]
Evidence:  -3175.5371329967847


#### Divide and conquer

In [9]:
def divide_and_conquer(mcm, final_mcm):
    best_mcm = mcm
    best_ev = model.calc_log_evidence(mcm)
    print('start: ', mcm, best_ev)
    mcm.append([])
    while True:
        for i in range(len(mcm[0])):
            new_mcm = [mcm[i][:] for i in range(len(mcm))]
            new_mcm[0] = mcm[0][:i] + mcm[0][i+1:]
            new_mcm[1].append(mcm[0][i])
            ev = model.calc_log_evidence(new_mcm)
            print(new_mcm, ev)
            if ev > best_ev:
                best_mcm = [new_mcm[i][:] for i in range(len(new_mcm))]
                best_ev = ev
            new_mcm[1].pop()

        if best_mcm != mcm:
            mcm = [best_mcm[i][:] for i in range(len(best_mcm))]
        else:
            break
        
    if len(best_mcm[1]) == 0:
        final_mcm.append(best_mcm[0])
    else:
        for subpart in best_mcm:
            if len(subpart) != 1:
                divide_and_conquer([subpart], final_mcm)
            else:
                final_mcm.append(subpart)
            

In [18]:
model = mcm('../data/SC_voting/US_SupremeCourt_n9_N895.txt')
model.find_best_im(max_interactions=2)
current_mcm = model.mcms[0]
best_mcm = []
divide_and_conquer(current_mcm, best_mcm)

start:  [[0, 1, 2, 3, 4, 5, 6, 7, 8]] -3305.546575913694
[[1, 2, 3, 4, 5, 6, 7, 8], [0]] -3196.1534648768757
[[0, 2, 3, 4, 5, 6, 7, 8], [1]] -3232.967206417273
[[0, 1, 3, 4, 5, 6, 7, 8], [2]] -3276.2105695410914
[[0, 1, 2, 4, 5, 6, 7, 8], [3]] -3238.7848195356487
[[0, 1, 2, 3, 5, 6, 7, 8], [4]] -3286.9193213087083
[[0, 1, 2, 3, 4, 6, 7, 8], [5]] -3303.8360376666033
[[0, 1, 2, 3, 4, 5, 7, 8], [6]] -3251.125221704139
[[0, 1, 2, 3, 4, 5, 6, 8], [7]] -3298.8634162189564
[[0, 1, 2, 3, 4, 5, 6, 7], [8]] -3270.614684726284
[[2, 3, 4, 5, 6, 7, 8], [0, 1]] -3185.0411097860288
[[1, 3, 4, 5, 6, 7, 8], [0, 2]] -3210.385755991394
[[1, 2, 4, 5, 6, 7, 8], [0, 3]] -3175.5371329967847
[[1, 2, 3, 5, 6, 7, 8], [0, 4]] -3202.06541746028
[[1, 2, 3, 4, 6, 7, 8], [0, 5]] -3248.4447409040863
[[1, 2, 3, 4, 5, 7, 8], [0, 6]] -3194.077768326539
[[1, 2, 3, 4, 5, 6, 8], [0, 7]] -3239.0712266890296
[[1, 2, 3, 4, 5, 6, 7], [0, 8]] -3216.9646662422515
[[2, 4, 5, 6, 7, 8], [0, 3, 1]] -3202.7987448731014
[[1, 4, 5, 6, 

In [15]:
best_mcm

[[1, 2, 4, 5, 6, 7, 8], [0, 3]]

In [16]:
model = mcm('../data/SC_voting/US_SupremeCourt_n9_N895.txt')
current_mcm = model.mcms[0]
best_mcm = []
divide_and_conquer(current_mcm, best_mcm)

start:  [[0, 1, 2, 3, 4, 5, 6, 7, 8]] -3305.54657591369
[[1, 2, 3, 4, 5, 6, 7, 8], [0]] -3506.473953479855
[[0, 2, 3, 4, 5, 6, 7, 8], [1]] -3423.151089359927
[[0, 1, 3, 4, 5, 6, 7, 8], [2]] -3505.3254342779846
[[0, 1, 2, 4, 5, 6, 7, 8], [3]] -3515.4197429054993
[[0, 1, 2, 3, 5, 6, 7, 8], [4]] -3500.4252510768592
[[0, 1, 2, 3, 4, 6, 7, 8], [5]] -3553.896470753466
[[0, 1, 2, 3, 4, 5, 7, 8], [6]] -3503.42222728104
[[0, 1, 2, 3, 4, 5, 6, 8], [7]] -3567.686532154147
[[0, 1, 2, 3, 4, 5, 6, 7], [8]] -3539.81806525089


In [17]:
best_mcm

[[0, 1, 2, 3, 4, 5, 6, 7, 8]]

### Generating samples from spin model with discrete states

In [20]:
q = 3
n = 2

In [21]:
# Construct random array for parameters (range?)
g = np.random.uniform(low=-1, high=1, size=q**n)

In [22]:
def construct_s(q):
    S = np.ones((q, q), dtype=complex)
    for i in range(q):
        for j in range(i, q):
            S[i,j] = np.exp(-((i*j)*2j*np.pi) / q)
            S[j,i] = S[i,j]
    return S

In [23]:
S = construct_s(q)
S_matrix = np.copy(S)
for _ in range(n-1):
    S_matrix = np.kron(S_matrix, S)

In [24]:
S

array([[ 1. +0.j       ,  1. +0.j       ,  1. +0.j       ],
       [ 1. +0.j       , -0.5-0.8660254j, -0.5+0.8660254j],
       [ 1. +0.j       , -0.5+0.8660254j, -0.5-0.8660254j]])

In [25]:
S_matrix

array([[ 1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
         1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
         1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j],
       [ 1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
         1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
         1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j],
       [ 1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j,
         1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j,
         1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j],
       [ 1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
        -0.5-8.66025404e-01j, -0.5-8.66025404e-01j, -0.5-8.66025404e-01j,
        -0.5+8.66025404e-01j, -0.5+8.66025404e-01j, -0.5+8.66025404e-01j],
       [ 1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
        -0.5-8.66025404e-01j, -0.5

In [90]:
p = np.exp(S_matrix @ g)

In [92]:
# Imaginary probabilities
p /= np.sum(p)
p

array([ 0.02305325-6.16336856e-17j,  0.00243038-9.64589741e-03j,
        0.00243038+9.64589741e-03j, -0.00350986+1.62851899e-02j,
        0.01281934+3.84899618e-03j,  0.47673352+8.51890406e-02j,
       -0.00350986-1.62851899e-02j,  0.47673352-8.51890406e-02j,
        0.01281934-3.84899618e-03j])